In [1]:
import numpy as np 
import matplotlib.pyplot as plt
from scipy.stats import norm

In [33]:
#MC Methods for the option price

def MC_option_price(n, S, T, K, r, sigma):
    Gt = np.random.normal(0,1,n)
    Wt = np.sqrt(T) * Gt

    St = S * np.exp(sigma * Wt + (r-(sigma**2)/2)*T)

    payoffs = np.maximum(St-K,0)

    prix = np.exp(-r*T)*np.mean(payoffs)

    upper_bound = prix + 1.96*np.std(payoffs)/np.sqrt(n)
    lower_bound = prix - 1.96*np.std(payoffs)/np.sqrt(n)

    print('Upper bound')
    print(upper_bound)
    print("-----------------------")
    print("price with 95% confidence interval")
    print(prix)
    print("-----------------------")
    print('Lower bound')
    print(lower_bound)
    
    return prix


MC_price = MC_option_price(10000000,100,1,110,0.05,0.2)

Upper bound
6.047319211169789
-----------------------
price with 95% confidence interval
6.039736838654045
-----------------------
Lower bound
6.0321544661383015


In [ ]:
#Formule fermée de Black Scholes

def BS_option_price(S,T,K,r,sigma): 
    d1 = (np.log(S/K) + (r+0.5*(sigma**2))*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)

    call_price = S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

    return call_price

BS_price = BS_option_price(100,1,110,0.05,0.2)
print(BS_price)



6.040088129724239


In [36]:
error = abs(MC_price - BS_price)
print(error)

0.00035129107019393757


In [10]:
"""
asian_option_mc.py

Script Python exécutable pour pricer une option asiatique (moyenne arithmétique)
via simulation Monte-Carlo. Usage en ligne de commande et paramètres configurables.

Fonctionnalités :
- simulation vectorisée de trajectoires GBM
- moyenne arithmétique discrète sur M dates
- call / put
- variates antithétiques optionnels
- sortie du prix estimé, erreur-type et intervalle de confiance à 95%

Exemple :
    python asian_option_mc.py --S0 100 --K 100 --r 0.05 --sigma 0.2 --T 1 --M 50 --N 200000 --type call --antithetic

"""

import argparse
import time
import numpy as np
from math import sqrt


def simulate_asian_mc(S0, K, r, sigma, T, M=50, N=100000, option_type="call", antithetic=True, seed=None):
    """Monte-Carlo pricing for a discretely sampled arithmetic Asian option.

    Returns dict {price, std_error, ci_low, ci_high, samples}
    """
    rng = np.random.default_rng(seed)
    dt = T / M
    discount = np.exp(-r * T)

    # Generate standard normals
    Z = rng.standard_normal(size=(N, M))
    if antithetic:
        Z_full = np.vstack([Z, -Z])
    else:
        Z_full = Z
    total_paths = Z_full.shape[0]

    drift = (r - 0.5 * sigma ** 2) * dt
    diffusion = sigma * np.sqrt(dt) * Z_full
    log_increments = drift + diffusion
    log_paths = np.cumsum(log_increments, axis=1)

    S_paths = S0 * np.exp(log_paths)
    arith_mean = S_paths.mean(axis=1)

    if option_type.lower() == "call":
        payoffs = np.maximum(arith_mean - K, 0.0)
    elif option_type.lower() == "put":
        payoffs = np.maximum(K - arith_mean, 0.0)
    else:
        raise ValueError("option_type must be 'call' or 'put'")

    discounted = discount * payoffs
    price_est = float(discounted.mean())
    std_error = float(discounted.std(ddof=1) / sqrt(total_paths))

    # 95% confidence interval
    z95 = 1.96
    ci_low = price_est - z95 * std_error
    ci_high = price_est + z95 * std_error

    return {
        "price": price_est,
        "std_error": std_error,
        "ci_low": ci_low,
        "ci_high": ci_high,
        "samples": int(total_paths),
    }


def parse_args():
    p = argparse.ArgumentParser(description="Monte-Carlo Asian option pricer")
    p.add_argument("--S0", type=float, default=100.0, help="Spot price")
    p.add_argument("--K", type=float, default=100.0, help="Strike")
    p.add_argument("--r", type=float, default=0.05, help="Risk-free rate (annual, cont.)")
    p.add_argument("--sigma", type=float, default=0.2, help="Volatility (annual)")
    p.add_argument("--T", type=float, default=1.0, help="Time to maturity (years)")
    p.add_argument("--M", type=int, default=50, help="Number of observation dates")
    p.add_argument("--N", type=int, default=100000, help="Number of Monte-Carlo base paths")
    p.add_argument("--type", choices=["call", "put"], default="call", help="Option type")
    p.add_argument("--antithetic", action="store_true", help="Use antithetic variates")
    p.add_argument("--seed", type=int, default=None, help="RNG seed (optional)")
    return p.parse_args()


def main():
    args = parse_args()
    params = vars(args)

    print("Asian option Monte-Carlo pricer")
    print("Parameters:")
    for k, v in params.items():
        print(f"  {k} = {v}")

    t0 = time.time()
    res = simulate_asian_mc(
        S0=args.S0,
        K=args.K,
        r=args.r,
        sigma=args.sigma,
        T=args.T,
        M=args.M,
        N=args.N,
        option_type=args.type,
        antithetic=args.antithetic,
        seed=args.seed,
    )
    t1 = time.time()

    print("\nResult:")
    print(f"  Price estimate = {res['price']:.6f}")
    print(f"  Std error     = {res['std_error']:.6f}")
    print(f"  95% CI        = [{res['ci_low']:.6f}, {res['ci_high']:.6f}]")
    print(f"  Samples       = {res['samples']}")
    print(f"  Elapsed time  = {t1-t0:.3f} s")


if __name__ == "__main__":
    # ⚠ Ici pas d'argparse
    params = {
        "S0": 100.0,
        "K": 100.0,
        "r": 0.05,
        "sigma": 0.2,
        "T": 1.0,
        "M": 50,
        "N": 200000,
        "option_type": "call",
        "antithetic": False,
        "seed": 2025,
    }

    print("Asian option Monte-Carlo pricer")
    for k, v in params.items():
        print(f"  {k} = {v}")

    t0 = time.time()
    res = simulate_asian_mc(**params)
    t1 = time.time()

    print("\nResult:")
    print(f"  Price estimate = {res['price']:.6f}")
    print(f"  Std error     = {res['std_error']:.6f}")
    print(f"  95% CI        = [{res['ci_low']:.6f}, {res['ci_high']:.6f}]")
    print(f"  Samples       = {res['samples']}")
    print(f"  Elapsed time  = {t1-t0:.3f} s")

Asian option Monte-Carlo pricer
  S0 = 100.0
  K = 100.0
  r = 0.05
  sigma = 0.2
  T = 1.0
  M = 50
  N = 200000
  option_type = call
  antithetic = False
  seed = 2025

Result:
  Price estimate = 5.871987
  Std error     = 0.018135
  95% CI        = [5.836442, 5.907532]
  Samples       = 200000
  Elapsed time  = 0.109 s


In [18]:
# Option à barrière 

def MC_barrier_option(S,T,K,B,n,N,r,sigma) :

    rng = np.random.default_rng(42)
    dt = T / n
    discount = np.exp(-r * T)

    Z = rng.standard_normal(size=(N, n))

    drift = (r - 0.5 * sigma ** 2) * dt
    diffusion = sigma * np.sqrt(dt) * Z
    log_increments = drift + diffusion
    log_paths = np.cumsum(log_increments, axis=1)

    S_paths = S * np.exp(log_paths)
    indicator = 1*(np.max(S_paths, axis=1) > B)

    payoff  = payoff = np.maximum(S_paths[:, n-1] - K, 0) * indicator

    price = np.mean(payoff) * discount

    std_error = float(payoff.std(ddof=1) / sqrt(Z.shape[0]))
    
    z95 = 1.96
    ci_low = price - z95 * std_error
    ci_high = price + z95 * std_error

    print('Upper Bound : ', ci_high)
    print('----------------------')
    print('MC Price :', price)
    print('----------------------')
    print('Lower bound :', ci_low)

    return price

MC_barrier_option(100,1,110,115,252,1000000,0.05,0.2)

Upper Bound :  6.044486175945719
----------------------
MC Price : 6.02043803760015
----------------------
Lower bound : 5.996389899254581


np.float64(6.02043803760015)

In [23]:

def BSCallAntithetic(S0, K, T, r, sigma, M, seed=None):
    """
    Price a European call option using Monte Carlo with and without antithetic variates.
    
    Parameters
    ----------
    S0 : float
        Spot price
    K : float
        Strike
    T : float
        Time to maturity
    r : float
        Risk-free rate (annual, continuous)
    sigma : float
        Volatility (annual)
    M : int
        Number of Monte Carlo samples (should be even for antithetic)
    seed : int, optional
        RNG seed

    Returns
    -------
    tuple : (price_MC, price_antithetic, RMSE_MC, RMSE_antithetic)
    """
    rng = np.random.default_rng(seed)
    t0 = time.time()
    
    # Standard normal samples
    X = rng.standard_normal(M)
    
    # Without antithetic
    C = np.exp(-r*T) * np.maximum(S0 * np.exp(sigma*np.sqrt(T)*X + (r - 0.5*sigma**2)*T) - K, 0)
    
    # Antithetic variates (split first half)
    G = X[:M//2]
    hatC = (np.exp(-r*T) * np.maximum(S0 * np.exp(sigma*np.sqrt(T)*G + (r - 0.5*sigma**2)*T) - K, 0) +
            np.exp(-r*T) * np.maximum(S0 * np.exp(-sigma*np.sqrt(T)*G + (r - 0.5*sigma**2)*T) - K, 0))
    
    # Estimates
    price = np.mean(C)
    hatprice = np.sum(hatC)/M
    
    VarEst = np.var(C, ddof=1)
    hatVarEst = np.var(hatC, ddof=1)
    
    RMSE = np.sqrt(VarEst)/np.sqrt(M)
    hatRMSE = np.sqrt(hatVarEst)/np.sqrt(M)
    
    t1 = time.time()
    
    print(f"Elapsed time: {t1-t0:.4f} s")
    
    return price, hatprice, RMSE, hatRMSE

# Exemple d'utilisation
price_MC, price_hat, RMSE_MC, RMSE_hat = BSCallAntithetic(S0=100, K=110, T=1, r=0.05, sigma=0.2, M=1000000, seed=42)
print(f"Price MC: {price_MC:.6f}, Price antithetic: {price_hat:.6f}")
print(f"RMSE MC: {RMSE_MC:.6f}, RMSE antithetic: {RMSE_hat:.6f}")


Elapsed time: 0.0339 s
Price MC: 6.044039, Price antithetic: 6.045966
RMSE MC: 0.011648, RMSE antithetic: 0.014090
